<a href="https://colab.research.google.com/github/nsingh216/stock_market_forecasting/blob/master/Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip3 install pmdarima
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
import matplotlib.pyplot as plt




     |████████████████████████████████| 921kB 4.9MB/s 


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_path = "/content/drive/My Drive/Colab Notebooks/"

df = pd.read_csv(data_path + "cooked_v3.csv")
print(df.head())
print(df.CalcDateTime.min())
print(df.CalcDateTime.max())

# tickers and their traded volumes
volume_series = df.groupby('Mnemonic')['TradedVolume'].sum()
volume_series.sort_values(ascending=False, inplace=True)
print(volume_series)

df.head()
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)


start_time = datetime.now()

          CalcDateTime Mnemonic  MinPrice  MaxPrice  StartPrice  EndPrice  \
0  2017-07-03 08:00:00      SNH     4.450     4.456       4.456     4.450   
1  2017-07-03 08:01:00      SNH     4.450     4.456       4.456     4.450   
2  2017-07-03 08:02:00      SNH     4.450     4.456       4.456     4.450   
3  2017-07-03 08:03:00      SNH     4.457     4.457       4.457     4.457   
4  2017-07-03 08:04:00      SNH     4.457     4.457       4.457     4.457   

   HasTrade  TradedVolume  NumberOfTrades  
0       1.0        5505.0             9.0  
1       0.0           0.0             0.0  
2       0.0           0.0             0.0  
3       1.0         899.0             2.0  
4       0.0           0.0             0.0  
2017-07-03 08:00:00
2018-05-15 20:00:00
Mnemonic
SNH     4.182139e+09
DBK     2.346360e+09
EOAN    1.883320e+09
DTE     1.695683e+09
CBK     1.575910e+09
RWE     8.374443e+08
IFX     7.630757e+08
SVAB    6.026515e+08
LHA     5.934044e+08
DAI     4.894308e+08
O2D     4.2069

In [0]:

for ticker in volume_series.index:
    ticker_data = df[df['Mnemonic'] == ticker]
    ticker_data.reset_index(inplace=True)

    X = pd.to_numeric(pd.to_datetime(ticker_data['CalcDateTime']))
    X = X.values.reshape(-1, 1)

    Y = ticker_data['StartPrice']
    print("Ticker " + ticker + " has " + str(len(X)) + " records.")


    ticker_data['CalcDateTime'] = pd.to_datetime(ticker_data['CalcDateTime'])
    # ticker_data = ticker_data[['CalcDateTime', 'StartPrice']]
    ticker_data.set_index(ticker_data['CalcDateTime'], inplace=True)

    # decomp freq: https://www.cbcity.de/timeseries-decomposition-in-python-with-statsmodels-and-pandas
    result = seasonal_decompose(ticker_data.StartPrice, freq=10080)
    trend = result.trend
    result.plot()
    plt.show()
    
    pm.plot_acf(ticker_data.StartPrice)
    print(pm.utils.acf(ticker_data.StartPrice))
    
    pm.plot_pacf(ticker_data.StartPrice)
    print(pm.utils.pacf(ticker_data.StartPrice))

    # predict last 15 values since arima on 25% of the data is taking too long
#     train_index = np.arange(0,len(X) - 15)
#     test_index = np.arange(len(X) - 15, len(X))
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]

#     stepwise_model = pm.auto_arima(ticker_data.StartPrice)
#     print(stepwise_model.summary())
#     print(stepwise_model.aic())



end_time = datetime.now()
c = end_time - start_time
print("Ran ARIMA in:" + str(c.total_seconds()))
